# Create `MBTiles`
In this notebook we will transforms vector type data into MBTiles using tippecanoe.

# Setup
## Library import
We import all the required Python libraries

In [149]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import subprocess
from dotenv import load_dotenv
import glob
from google.cloud import storage
from google.oauth2 import service_account
import ipyleaflet as ipyl

## Utils

**read_indicators**

In [150]:
def read_indicators(dataset, indicator, gdf_vectors, geo_type='Municipios'):

    gdf_ids = {'Provincias': ['CO_CCAA', 'CO_PROVINC'],
    'Comunidades autonomas': ['CO_CCAA'],
    'Municipios': ['CODIGOINE']}

    gdf_columns = {'Provincias': ['DS_CCAA', 'CO_CCAA', 'DS_PROVINC', 'CO_PROVINC'],
    'Comunidades autonomas': ['DS_CCAA', 'CO_CCAA'],
    'Municipios': ['NAMEUNIT', 'CODIGOINE']}

    geo_name = geo_type.lower().replace(' ', '_')

    df = pd.read_csv(f'../../datasets/processed/{dataset}/{dataset}_{geo_name}.csv')
    df = df.astype(dict(zip(gdf_ids[geo_type], [int]*len(gdf_ids[geo_type]))))
    gdf = df[(df['indicator'] == indicator)]
    gdf = gpd.GeoDataFrame(pd.merge(gdf, gdf_vectors[geo_type][gdf_columns[geo_type]+['geometry']].astype(dict(zip(gdf_ids[geo_type], [int]*len(gdf_ids[geo_type])))),\
         how='left', on=gdf_ids[geo_type]))

    gdf = gdf[gdf_columns[geo_type] + ['dataset', 'indicator', 'scenario', 'value', 'year', 'unit', 'geometry']]

    return gdf

**reorganize_data**

In [157]:
def reorganize_data(gdf, geo_type='Municipios'):
    gdf_id = {'Provincias': 'CO_PROVINC',
    'Comunidades autonomas': 'CO_CCAA',
    'Municipios': 'CODIGOINE'}

    gdf_columns = {'Provincias': ['DS_CCAA', 'CO_CCAA', 'DS_PROVINC', 'CO_PROVINC'],
    'Comunidades autonomas': ['DS_CCAA', 'CO_CCAA'],
    'Municipios': ['NAMEUNIT', 'CODIGOINE']}

    gdf_new = gpd.GeoDataFrame(columns=gdf_columns[geo_type]['dataset', 'indicator', 'unit', 'values', 'geometry'])
    for code_id in gdf[gdf_id[geo_type]].unique():
        LD = gdf[gdf[gdf_id[geo_type]] == code_id][['scenario', 'year', 'value']].to_dict('records')
        v = {k: {dic['year']: dic['value'] for dic in LD if dic['scenario'] == k} for k in ['rcp45', 'rcp85']}

        gdf_tmp = gdf[gdf[gdf_id[geo_type]] == code_id][gdf_columns[geo_type]].iloc[:1]
        gdf_tmp['dataset'] = gdf['dataset'].iloc[0]
        gdf_tmp['indicator'] = gdf['indicator'].iloc[0]
        gdf_tmp['unit'] = gdf['unit'].iloc[0]
        gdf_tmp['values'] = v
        gdf_tmp['geometry'] = gdf[gdf['CODIGOINE'] == code_id]['geometry'].iloc[0]

        gdf_new = pd.concat([gdf_new, gdf_tmp])

    return gdf_new

In [151]:
def vector_tile_processing(path, dataset, folder_name, gdf, geo_type='Municipios', pbf_format=True, upload_to_GCS=True):
    geo_name = geo_type.lower().replace(' ', '_')
    
    # Save GeoDataFrame as `GeoJSON`
    gdf.to_file(f"{path}/{folder_name}/{geo_name}.json", driver="GeoJSON")

    # Create MBTiles
    print('Creating MBTiles')
    layer_name = name
    source_path = f"{path}/{folder_name}/{geo_name}.json"
    mbtil_path = f"{path}/{folder_name}/{geo_name}.mbtiles"
    create_mbtiles(source_path, mbtil_path, layer_name, opts="-zg --drop-densest-as-needed --extend-zooms-if-still-dropping --force --read-parallel")

    # Convert from `MBTiles` to `pbf` in {z}/{x}/{y}.pbf format
    print('Converting from MBTiles to pbf')
    if pbf_format:
        output_path = f"{path}/{folder_name}/{geo_name}"
        mbtile_to_pbf(mbtil_path, output_path)

    # Upload `pbf` files to GCS
    print('Uploading pbf files to GCS')
    if upload_to_GCS:
        load_dotenv()

        bucket_name = 'ecf-agricultural-climate-impact'
        local_path = f"{path}/{folder_name}/{geo_name}"
        destination_blob_path = f"MBTiles/{folder_name}/{geo_name}/"

        upload_local_directory_to_gcs(bucket_name, local_path, destination_blob_path)

**create_mbtiles**

In [81]:
def create_mbtiles(source_path, dest_path, layer_name, opts="-zg --drop-densest-as-needed --extend-zooms-if-still-dropping --force --read-parallel"):
    """
    Use tippecanoe to create a MBTILE at dest_path from source_path.
    layer_name is used for the name of the layer in the MBTILE.
    Regex file path (/*.geojson) is supported for source_path.
    """
    cmd = f"tippecanoe -o {dest_path} -l {layer_name} {opts} {source_path}"
    print(f"Processing: {cmd}")
    r = subprocess.call(cmd, shell=True)
    if r == 0:
        print("Task created")
    else:
        print("Task failed")
    print("Finished processing")

**mbtile_to_pbf**

In [82]:
def mbtile_to_pbf(input_mbtiles, output_path):
    """
    Use mb-util to convert mbtiles to to pbf in z/x/y.pbf form.
    To export MBTiles to disk, specify a directory that does not yet exist.
    """
    cmd = f"mb-util {input_mbtiles} {output_path} --image_format=pbf"
    print(f"Processing: {cmd}")
    r = subprocess.call(cmd, shell=True)
    if r == 0:
        print("Task created")
    else:
        print("Task failed")
    print("Finished processing")

**upload_local_directory_to_gcs**

In [83]:
def upload_local_directory_to_gcs(bucket_name, local_path, destination_blob_path):
    """Uploads a directory to the bucket."""
    
    credentials_dict = {
          "type": "service_account",
          "project_id": os.getenv("PROJECT_ID"),
         "private_key_id": os.getenv("PRIVATE_KEY_ID"),
          "private_key":os.getenv("PRIVATE_KEY"),
         "client_email": os.getenv("CLIENT_EMAIL"),
          "token_uri":os.getenv("TOKEN_URI"),   
    } 
    

    credentials = service_account.Credentials.from_service_account_info(credentials_dict)
    storage_client = storage.Client(project='project_id', credentials=credentials)
    
    bucket = storage_client.bucket(bucket_name)
    rel_paths = glob.glob(local_path + '/**', recursive=True)

    for local_file in rel_paths:
        remote_path = f'{destination_blob_path}{"/".join(local_file.split(os.sep)[len(local_path.split(os.sep))-1:])}'
        if os.path.isfile(local_file):
            blob = bucket.blob(remote_path)
            print(
                "File {} uploaded to {}.".format(
                    local_file, remote_path
                )
            )
            blob.upload_from_filename(local_file)

**vector_tile_processing**

In [158]:
def vector_tile_processing(path, dataset, folder_name, gdf, geo_type='Municipios', pbf_format=True, upload_to_GCS=True):
    geo_name = geo_type.lower().replace(' ', '_')
    
    # Save GeoDataFrame as `GeoJSON`
    gdf.to_file(f"{path}/{folder_name}/{geo_name}.json", driver="GeoJSON")

    # Create MBTiles
    print('Creating MBTiles')
    layer_name = name
    source_path = f"{path}/{folder_name}/{geo_name}.json"
    mbtil_path = f"{path}/{folder_name}/{geo_name}.mbtiles"
    create_mbtiles(source_path, mbtil_path, layer_name, opts="-zg --drop-densest-as-needed --extend-zooms-if-still-dropping --force --read-parallel")

    # Convert from `MBTiles` to `pbf` in {z}/{x}/{y}.pbf format
    print('Converting from MBTiles to pbf')
    if pbf_format:
        output_path = f"{path}/{folder_name}/{geo_name}"
        mbtile_to_pbf(mbtil_path, output_path)

    # Upload `pbf` files to GCS
    print('Uploading pbf files to GCS')
    if upload_to_GCS:
        load_dotenv()

        bucket_name = 'ecf-agricultural-climate-impact'
        local_path = f"{path}/{folder_name}/{geo_name}"
        destination_blob_path = f"MBTiles/{folder_name}/{geo_name}/"

        upload_local_directory_to_gcs(bucket_name, local_path, destination_blob_path)

**all_processing**

In [166]:
def all_processing(dataset, indicator, gdf_vectors, path, folder_name, geo_types = ['Provincias', 'Municipios']):
    for geo_type in geo_types:
        print(geo_type)
        # Read data
        gdf = read_indicators(dataset, indicator, gdf_vectors, geo_type)

        # Reorganice data
        gdf = reorganice_data(gdf, geo_type) 

        # Create `MBTiles` and upload to GCS in {z}/{x}/{y}.pbf format
        vector_tile_processing(path, dataset, folder_name, gdf, geo_type, pbf_format=True, upload_to_GCS=True)

# Data import

## Comunidades autonomas

In [89]:
comunidades = gpd.read_file(f'../../datasets/processed/comunidades.geojson')
comunidades.sort_values(['CO_CCAA'], inplace = True)

## Provincias

In [90]:
provincias = gpd.read_file(f'../../datasets/raw/georegions/Provincias/Provincias.shp')
provincias.sort_values(['CO_CCAA', 'CO_PROVINC'], inplace = True)

## Municipios de España

In [91]:
municipios = gpd.read_file(f'../../datasets/raw/georegions/Municipios/Municipios_IGN.shp')
municipios.sort_values(['CODNUT1', 'CODNUT2', 'CODNUT3', 'CODIGOINE'], inplace = True)

# Remove Canarias, Ceuta, and Melilla
municipios = municipios[~municipios['CODNUT2'].isin(['ES70', 'ES63', 'ES64'])]
municipios = municipios.reset_index(drop=True)

**Put all vector data together** 

In [92]:
gdf_vectors = {'Comunidades autonomas': comunidades,
"Provincias": provincias,
"Municipios": municipios}

# Create tiles

## Aumento de temperaturas proyectadas por municipio
**Read data**

In [153]:
dataset = 'climate_indicators'
indicator = 'mean_Tmean_Yearly_change'
path = '../../datasets/processed/MBTiles'
folder_name = 'Aumento_temperaturas'

all_processing(dataset, indicator, gdf_vectors, path, folder_name, geo_types)

## Duración de las sequías

In [161]:
dataset = 'climate_indicators'
indicator = 'dry-spells_maximum-length'
path = '../../datasets/processed/MBTiles'
folder_name = 'Duracion_sequias'
geo_types = ['Comunidades autonomas', 'Provincias', 'Municipios']

all_processing(dataset, indicator, gdf_vectors, path, folder_name, geo_types)

## Mapa de cultivos
**Read data**

In [84]:
gdf_corine = gpd.read_file('../../datasets/raw/crops/CLC2018_GDB/CLC2018_ES.gdb', driver="FileGDB", layer='CLC18_ES')
# Re-Project
gdf_corine = gdf_corine.to_crs("EPSG:4326")
gdf_corine = gdf_corine.set_crs("EPSG:4326")

**Reorganice data**

In [109]:
dic = {'Cereal': ['211', '212'],
'Olivar': ['223'],
'Dehesa': ['244'],
'Viñedo': ['221']}

gdf = gpd.GeoDataFrame(columns=['ID', 'value', 'geometry'])

for crop in dic.keys():
    print(crop)
    gdf_tmp = gdf_corine[gdf_corine['CODE_18'].isin(dic[crop])].copy()
    gdf_tmp['value'] = crop
    gdf_tmp = gdf_tmp[['ID', 'value', 'geometry']]
    
    gdf  = pd.concat([gdf, gdf_tmp])

gdf['dataset'] = 'Mapa de cultivos'
gdf['indicator'] = 'Localización cultivo'

gdf = gdf[['ID', 'dataset', 'indicator', 'value', 'geometry']]

gdf.head()

Cereal
Olivar
Dehesa
Viñedo


,ID,dataset,indicator,value,geometry
16379,16382,Mapa de cultivos,Localización cultivo,Cereal,"MULTIPOLYGON (((-5.64246 36.12721, -5.64226 36..."
16380,16383,Mapa de cultivos,Localización cultivo,Cereal,"MULTIPOLYGON (((-5.50050 36.17831, -5.50045 36..."
16381,16384,Mapa de cultivos,Localización cultivo,Cereal,"MULTIPOLYGON (((-5.50964 36.18659, -5.50953 36..."
16382,16385,Mapa de cultivos,Localización cultivo,Cereal,"MULTIPOLYGON (((-5.69228 36.18296, -5.69232 36..."
16383,16386,Mapa de cultivos,Localización cultivo,Cereal,"MULTIPOLYGON (((-5.46238 36.20897, -5.46238 36..."


#### **Create `MBTiles` and upload to GCS in {z}/{x}/{y}.pbf format**

In [110]:
path = '../../datasets/processed/MBTiles'
name = 'Mapa_cultivos'

vector_tile_processing(path, name, gdf, pbf_format=True, upload_to_GCS=True)

Processing: tippecanoe -o ../../datasets/processed/MBTiles/Mapa_cultivos.mbtiles -l Mapa_cultivos -zg --drop-densest-as-needed --extend-zooms-if-still-dropping --force --read-parallel ../../datasets/processed/MBTiles/Mapa_cultivos.json


../../datasets/processed/MBTiles/Mapa_cultivos.json:1566: Reached EOF without all containers being closed
In JSON object {"type":"FeatureCollection","features":[]}
../../datasets/processed/MBTiles/Mapa_cultivos.json:1671: Found ] at top level
44488 features, 189477529 bytes of geometry, 548299 bytes of separate metadata, 318898 bytes of string pool
Choosing a maxzoom of -z5 for features about 9226 feet (2812 meters) apart
Choosing a maxzoom of -z12 for resolution of about 86 feet (26 meters) within features
tile 5/15/12 size is 965103 with detail 12, >500000    
Going to try keeping the sparsest 46.63% of the features to make it fit
tile 5/15/12 size is 509947 with detail 12, >500000    
Going to try keeping the sparsest 41.15% of the features to make it fit
tile 6/31/24 size is 1255285 with detail 12, >500000    
Going to try keeping the sparsest 35.85% of the features to make it fit
tile 6/31/23 size is 624460 with detail 12, >500000    
Going to try keeping the sparsest 72.06% of th

Task created
Finished processing
Processing: mb-util ../../datasets/processed/MBTiles/Mapa_cultivos.mbtiles ../../datasets/processed/MBTiles/Mapa_cultivos --image_format=pbf


DEBUG:mbutil.util:Exporting MBTiles to disk
DEBUG:mbutil.util:../../datasets/processed/MBTiles/Mapa_cultivos.mbtiles --> ../../datasets/processed/MBTiles/Mapa_cultivos
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1 / 11355 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:2 / 11355 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:3 / 11355 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:4 / 11355 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:5 / 11355 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:6 / 11355 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:7 / 11355 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:8 / 11355 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:9 / 11355 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:10 / 11355 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:11 / 11355 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:12 / 11355 tiles exported
D

Task created
Finished processing
File ../../datasets/processed/MBTiles/Mapa_cultivos/12/1962/1501.pbf uploaded to MBTiles/Mapa_cultivos/12/1962/1501.pbf.
File ../../datasets/processed/MBTiles/Mapa_cultivos/12/1962/1522.pbf uploaded to MBTiles/Mapa_cultivos/12/1962/1522.pbf.
File ../../datasets/processed/MBTiles/Mapa_cultivos/12/1962/1508.pbf uploaded to MBTiles/Mapa_cultivos/12/1962/1508.pbf.
File ../../datasets/processed/MBTiles/Mapa_cultivos/12/1962/1556.pbf uploaded to MBTiles/Mapa_cultivos/12/1962/1556.pbf.
File ../../datasets/processed/MBTiles/Mapa_cultivos/12/1962/1513.pbf uploaded to MBTiles/Mapa_cultivos/12/1962/1513.pbf.
File ../../datasets/processed/MBTiles/Mapa_cultivos/12/1962/1588.pbf uploaded to MBTiles/Mapa_cultivos/12/1962/1588.pbf.
File ../../datasets/processed/MBTiles/Mapa_cultivos/12/1962/1586.pbf uploaded to MBTiles/Mapa_cultivos/12/1962/1586.pbf.
File ../../datasets/processed/MBTiles/Mapa_cultivos/12/1962/1502.pbf uploaded to MBTiles/Mapa_cultivos/12/1962/1502.pbf.

## Proyecciones de rendimiento del olivo.

In [180]:
dataset = 'olive_grove_indicators'
indicator = 'yield_change_per'
path = '../../datasets/processed/MBTiles'
folder_name = 'Proyecciones_rendimiento'
geo_types = ['Comunidades autonomas', 'Provincias', 'Municipios']

In [181]:
all_processing(dataset, indicator, gdf_vectors, path, folder_name, geo_types)

Comunidades autonomas
Creating MBTiles
Processing: tippecanoe -o ../../datasets/processed/MBTiles/Proyecciones_rendimiento/comunidades_autonomas.mbtiles -l denominaciones_de_origen_protegidas -zg --drop-densest-as-needed --extend-zooms-if-still-dropping --force --read-parallel ../../datasets/processed/MBTiles/Proyecciones_rendimiento/comunidades_autonomas.json


../../datasets/processed/MBTiles/Proyecciones_rendimiento/comunidades_autonomas.json:2: JSON does not allow NaN
In JSON object {"type":"Feature","properties":{"DS_CCAA":"Cantabria","CO_CCAA":"6","dataset":"Olivar","indicator":"yield_change_per","unit":"%","values":{"rcp45":{"2041–2070":...}}}}
../../datasets/processed/MBTiles/Proyecciones_rendimiento/comunidades_autonomas.json:2: JSON does not allow NaN
../../datasets/processed/MBTiles/Proyecciones_rendimiento/comunidades_autonomas.json:2: JSON does not allow NaN
In JSON object {"type":"Feature","properties":{"DS_CCAA":"Pais Vasco","CO_CCAA":"15","dataset":"Olivar","indicator":"yield_change_per","unit":"%","values":{"rcp45":{"2041–2070":...}}}}
In JSON object {"type":"Feature","properties":{"DS_CCAA":"Principado de Asturias","CO_CCAA":"3","dataset":"Olivar","indicator":"yield_change_per","unit":"%","values":{"rcp45":{"2041–2070":...}}}}
../../datasets/processed/MBTiles/Proyecciones_rendimiento/comunidades_autonomas.json:2: JSON does no

Task created
Finished processing
Converting from MBTiles to pbf
Processing: mb-util ../../datasets/processed/MBTiles/Proyecciones_rendimiento/comunidades_autonomas.mbtiles ../../datasets/processed/MBTiles/Proyecciones_rendimiento/comunidades_autonomas --image_format=pbf


DEBUG:mbutil.util:Exporting MBTiles to disk
DEBUG:mbutil.util:../../datasets/processed/MBTiles/Proyecciones_rendimiento/comunidades_autonomas.mbtiles --> ../../datasets/processed/MBTiles/Proyecciones_rendimiento/comunidades_autonomas
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1 / 282 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:2 / 282 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:3 / 282 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:4 / 282 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:5 / 282 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:6 / 282 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:7 / 282 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:8 / 282 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:9 / 282 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:10 / 282 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:11 / 282 tiles exported
DEBUG:mbutil.util:flipping


Task created
Finished processing
Uploading pbf files to GCS
File ../../datasets/processed/MBTiles/Proyecciones_rendimiento/comunidades_autonomas/2/2/1.pbf uploaded to MBTiles/Proyecciones_rendimiento/comunidades_autonomas/comunidades_autonomas/2/2/1.pbf.
File ../../datasets/processed/MBTiles/Proyecciones_rendimiento/comunidades_autonomas/2/1/1.pbf uploaded to MBTiles/Proyecciones_rendimiento/comunidades_autonomas/comunidades_autonomas/2/1/1.pbf.
File ../../datasets/processed/MBTiles/Proyecciones_rendimiento/comunidades_autonomas/4/7/6.pbf uploaded to MBTiles/Proyecciones_rendimiento/comunidades_autonomas/comunidades_autonomas/4/7/6.pbf.
File ../../datasets/processed/MBTiles/Proyecciones_rendimiento/comunidades_autonomas/4/7/5.pbf uploaded to MBTiles/Proyecciones_rendimiento/comunidades_autonomas/comunidades_autonomas/4/7/5.pbf.
File ../../datasets/processed/MBTiles/Proyecciones_rendimiento/comunidades_autonomas/4/8/6.pbf uploaded to MBTiles/Proyecciones_rendimiento/comunidades_autonoma